# Instructions
To compile ASN.1 specs from file `messages.asn` into python classes in file `messages.py`, run the following:

```sh
python /usr/local/lib/python2.7/dist-packages/asn1ate/pyasn1gen.py messages.asn > messages.py
```
Then modify the generated `messages.py` by adding the `Logical` and `Query` commands into a loop:

```py
for _ in xrange(10):
    Logical.tagSet = univ.Sequence.tagSet.tagImplicitly(tag.Tag(tag.tagClassApplication, tag.tagFormatConstructed, 5))
    Logical.componentType = namedtype.NamedTypes(
        namedtype.NamedType('op', univ.Enumerated(namedValues=namedval.NamedValues(('and', 0), ('or', 1)))),
        namedtype.NamedType('l', Query()),
        namedtype.NamedType('r', Query())
    )
    Query.componentType = namedtype.NamedTypes(
        namedtype.NamedType('cmp', Comparison()),
        namedtype.NamedType('log', Logical())
    )
```

In [1]:
from pyasn1.type import univ
from pyasn1.type import char
from pyasn1.type import tag
from pyasn1.type import namedtype
from pyasn1.type import constraint
from pyasn1.type import namedval
from pyasn1.codec.ber import encoder
from pyasn1.codec.ber import decoder
from trustmessages import *
from trustmessages.trustdatabase import QtmDb, SLDb
import base64
import itertools

In [2]:
encode = lambda m: base64.b64encode(encoder.encode(m))
get_bytes = lambda e: str([ord(i) for i in encoder.encode(e)])
users = itertools.cycle(["a@x.com", "b@x.com", "c@x.com"])
services = itertools.cycle(["buyer", "seller", "letter", "renter"])
quantitative = itertools.cycle(xrange(1, 6))
qualitative = itertools.cycle(["distrust", "neutral", "trust"])

In [3]:
r = Assessment()
r["source"] = next(users)
r["target"] = next(users)
r["service"] = next(services)
r["date"] = 100
val = next(quantitative)
r["value"] = encoder.encode(univ.Integer(val))

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Assessment())
assert(r.prettyPrint() == decoded.prettyPrint())
assert(decoder.decode(decoded["value"], asn1Spec=univ.Integer())[0] == val)
print encode(r)

aR8TB2FAeC5jb20TB2JAeC5jb20TBWJ1eWVyAgFkAgEB


In [4]:
r = Assessment()
r["source"] = next(users)
r["target"] = next(users)
r["service"] = next(services)
r["date"] = 100
val = next(qualitative)
r["value"] = encoder.encode(char.PrintableString(val))

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Assessment())
assert(r.prettyPrint() == decoded.prettyPrint())
assert(decoder.decode(decoded["value"], asn1Spec=char.PrintableString())[0] == val)
print encode(r)

aScTB2NAeC5jb20TB2FAeC5jb20TBnNlbGxlcgIBZBMIZGlzdHJ1c3Q=


In [5]:
a_req = AssessmentRequest()
a_req["rid"] = 1
sq = Query()
sq["cmp"] = Comparison()
sq["cmp"]["op"] = "ge"
sq["cmp"]["value"] = Value()
sq["cmp"]["value"]["date"] = 80
a_req["query"] = sq

decoded, _ = decoder.decode(encoder.encode(a_req), asn1Spec=Message())
assert(a_req.prettyPrint() == decoded.getComponent().prettyPrint())
print encode(a_req)

ZgsCAQFkBgoBBUIBUA==


In [6]:
a_res = AssessmentResponse()
a_res["provider"] = Entity("ebay")
a_res["format"] = Format((1, 1, 1))
a_res["rid"] = 1
a_res["response"] = univ.SequenceOf(componentType=Assessment())

for i in xrange(2):
    a = Assessment()
    a["source"] = next(users)
    a["target"] = next(users)
    a["service"] = next(services)
    a["date"] = 1000
    a["value"] = encoder.encode(univ.Integer(5))
    a_res["response"].setComponentByPosition(i, a)

data, _ = decoder.decode(encoder.encode(a_res), asn1Spec=Message())
assert(data.getComponent() == a_res)
assert(data.getComponent().prettyPrint() == a_res.prettyPrint())
print encode(a_res)

Z1UTBGViYXkGAikBAgEBMEZpIRMHYkB4LmNvbRMHY0B4LmNvbRMGbGV0dGVyAgID6AIBBWkhEwdhQHguY29tEwdiQHguY29tEwZyZW50ZXICAgPoAgEF


In [7]:
t_req = TrustRequest()

t_req["rid"] = 5000

sq = Query()
sq["cmp"] = Comparison()
sq["cmp"]["op"] = "ge"
sq["cmp"]["value"] = Value()
sq["cmp"]["value"]["date"] = 80
t_req["query"] = sq

data, _ = decoder.decode(encoder.encode(t_req), asn1Spec=Message())
assert(data.getComponent() == t_req)
assert(data.getComponent().prettyPrint() == t_req.prettyPrint())
print encode(t_req)

YgwCAhOIZAYKAQVCAVA=


In [8]:
t_res = TrustResponse()
t_res["provider"] = Entity("ebay")
t_res["format"] = Format((1, 1, 1))
t_res["rid"] = 70000
t_res["response"] = univ.SequenceOf(componentType=Trust())

for i in xrange(2):
    t = Trust()
    t["target"] = next(users)
    t["service"] = next(services)
    t["date"] = 2000
    t["value"] = encoder.encode(char.PrintableString(next(qualitative)))
    t_res["response"].setComponentByPosition(i, t)

data, _ = decoder.decode(encoder.encode(t_res), asn1Spec=Message())

assert(data.getComponent() == t_res)
assert(data.getComponent().prettyPrint() == t_res.prettyPrint())
print encode(t_res)

Y04TBGViYXkGAikBAgMBEXAwPWgdEwdjQHguY29tEwVidXllcgICB9ATB25ldXRyYWxoHBMHYUB4LmNvbRMGc2VsbGVyAgIH0BMFdHJ1c3Q=


In [9]:
f_req = FormatRequest()
data, _ = decoder.decode(encoder.encode(f_req), asn1Spec=Message())
assert(data.getComponent() == f_req)
assert(data.getComponent().prettyPrint() == f_req.prettyPrint())
print encode(f_req)

QAA=


In [10]:
f_res = FormatResponse()
f_res["format"] = Format((1, 2, 3))
f_res["assessment"] = char.PrintableString("Here be an ASN.1 spec for assessment values")
f_res["trust"] = char.PrintableString("Here be an ASN.1 spec for trust values")

data, _ = decoder.decode(encoder.encode(f_res), asn1Spec=Message())
assert(data.getComponent() == f_res)
assert(data.getComponent().prettyPrint() == f_res.prettyPrint())
print encode(f_res)

YVkGAioDEytIZXJlIGJlIGFuIEFTTi4xIHNwZWMgZm9yIGFzc2Vzc21lbnQgdmFsdWVzEyZIZXJlIGJlIGFuIEFTTi4xIHNwZWMgZm9yIHRydXN0IHZhbHVlcw==


In [11]:
e = Fault()
e["value"] = "invalid-parameters"
e["message"] = "something went wrong!"

data, _ = decoder.decode(encoder.encode(e), asn1Spec=Message())

assert(data.getComponent() == e)
assert(data.getComponent().prettyPrint() == e.prettyPrint())
print encode(e)

ahoKAQATFXNvbWV0aGluZyB3ZW50IHdyb25nIQ==


In [12]:
e1 = Fault()
e1["value"] = "invalid-parameters"
e1["message"] = "a"

e2 = Fault()
e2["value"] = "internal-error"
e2["message"] = "b"

print get_bytes(e1)
print get_bytes(e2)


[106, 6, 10, 1, 0, 19, 1, 97]
[106, 6, 10, 1, 1, 19, 1, 98]


In [13]:
# Encode SL
bdu = SLDb().AssessmentClass()
bdu["b"] = 0.1
bdu["d"] = 0.2
bdu["u"] = 0.7

decoded, _ = decoder.decode(encoder.encode(bdu), asn1Spec=SLDb().AssessmentClass())
assert(bdu == decoded)
print encode(bdu)

MBUJBQMxRS0xCQUDMkUtMQkFAzdFLTE=


In [14]:
# encode assessment with SL
r = Assessment()
r["source"] = next(users)
r["target"] = next(users)
r["service"] = next(services)
r["date"] = 100
r["value"] = encoder.encode(bdu)

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Assessment())
assert(r == decoded)
print encode(r)

aTQTB2JAeC5jb20TB2NAeC5jb20TBmxldHRlcgIBZDAVCQUDMUUtMQkFAzJFLTEJBQM3RS0x


In [15]:
# decode SL from Java
bytes = base64.decodestring("MCEJCYDJDMzMzMzMzQkJgMoMzMzMzMzNCQmAzAszMzMzMzM=")
bdu, _ = decoder.decode(bytes, asn1Spec=SLDb().AssessmentClass())
assert bdu["b"] == 0.1
assert bdu["d"] == 0.2
assert bdu["u"] == 0.7

In [16]:
# decode Java's Assessment@SL
bytes = base64.decodestring("aToTBWFsaWNlEwNib2ITBnNlbGxlcgIBCjAhC" \
                            "QmAyQzMzMzMzM0JCYDKDMzMzMzMzQkJgMwLMzMzMzMz")
a, _ = decoder.decode(bytes, asn1Spec=Assessment())
bdu, _ = decoder.decode(a["value"], asn1Spec=SLDb().AssessmentClass())
assert bdu["b"] == 0.1
assert bdu["d"] == 0.2
assert bdu["u"] == 0.7

In [17]:
# decode Java's AssessmentResponse@SL
bytes = base64.decodestring("Z4GbEwdzb21ldG1zBgIpAQIBATCBiGlCEwVhbGljZRMLeW91QHlvdS5jb20TBnN" \
                            "lbGxlcgIBCjAhCQmAyQzMzMzMzM0JCYDKDMzMzMzMzQkJgMwLMzMzMzMzaUITBW" \
                            "FsaWNlEwt5b3VAeW91LmNvbRMGc2VsbGVyAgEKMCEJCYDJDMzMzMzMzQkJgMoMz" \
                            "MzMzMzNCQmAzAszMzMzMzM=")
ar, _ = decoder.decode(bytes, asn1Spec=Message())
for a in ar["assessment-response"]["response"]:
    bdu, _ = decoder.decode(a["value"], asn1Spec=SLDb().AssessmentClass())
    assert bdu["b"] == 0.1
    assert bdu["d"] == 0.2
    assert bdu["u"] == 0.7

In [18]:
# Decoding Java's AssessmentResponse with SL (coming from SLDb)
encoded = base64.b64decode("Z4IBAhMHc29tZXRtcwYCKQECAQEwge9pOhMDYm9iEwVhbGljZRM" \
                           "Gc2VsbGVyAgEkMCEJCYDLG4bTcQ12XQkJgMkTOR2AYsXTCQnAyQ" \
                           "FUa0SYn0dpOhMDYm9iEwVhbGljZRMGbGV0dGVyAgElMCEJCYDLG" \
                           "BCvLDtiTwkJgMgLWLx2NgTzCQmAyg0Icon7uiVpOhMDYm9iEwVh" \
                           "bGljZRMGcmVudGVyAgEmMCEJCYDNBy0nZcWi6QkJgMoH0mD/ubz" \
                           "DCQnAygE7nC3m1AtpORMDYm9iEwVhbGljZRMFYnV5ZXICAScwIQ" \
                           "kJgMsWp61/OSArCQmAygpmLBWsDfEJCYDKCEp46+GxuQ==")
data, _ = decoder.decode(encoded, asn1Spec=Message())

In [19]:
# encode QTM
qtm = QtmDb().AssessmentClass("very-good")
decoded, _ = decoder.decode(encoder.encode(qtm), asn1Spec=QtmDb().AssessmentClass())
assert(qtm == decoded)
print encode(qtm)

CgEE


In [20]:
# encode assessment with QTM
r = Assessment()
r["source"] = next(users)
r["target"] = next(users)
r["service"] = next(services)
r["date"] = 100
r["value"] = encoder.encode(qtm)

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Assessment())
assert(r == decoded)
print encode(r)

aSATB2FAeC5jb20TB2JAeC5jb20TBnJlbnRlcgIBZAoBBA==


In [21]:
# AssessmentResponse with QTM
a_res = AssessmentResponse()
a_res["provider"] = Entity("ebay")
a_res["format"] = Format((1, 1, 1))
a_res["rid"] = 1
a_res["response"] = univ.SequenceOf(componentType=Assessment())

for i in xrange(2):
    qtm = QtmDb().AssessmentClass("very-good")
    
    a = Assessment()
    a["source"] = next(users)
    a["target"] = next(users)
    a["service"] = next(services)
    a["date"] = 1000
    a["value"] = encoder.encode(qtm)
    a_res["response"].setComponentByPosition(i, a)

data, _ = decoder.decode(encoder.encode(a_res), asn1Spec=Message())
assert(data.getComponent() == a_res)
assert(data.getComponent().prettyPrint() == a_res.prettyPrint())
print encode(a_res)

Z1QTBGViYXkGAikBAgEBMEVpIBMHY0B4LmNvbRMHYUB4LmNvbRMFYnV5ZXICAgPoCgEEaSETB2JAeC5jb20TB2NAeC5jb20TBnNlbGxlcgICA+gKAQQ=


In [22]:
# AssessmentResponse with SL
a_res = AssessmentResponse()
a_res["provider"] = Entity("ebay")
a_res["format"] = Format((1, 1, 1))
a_res["rid"] = 1
a_res["response"] = univ.SequenceOf(componentType=Assessment())

for i in xrange(2):
    bdu = SLDb().AssessmentClass()
    bdu["b"] = 0.1
    bdu["d"] = 0.2
    bdu["u"] = 0.7
    
    a = Assessment()
    a["source"] = next(users)
    a["target"] = next(users)
    a["service"] = next(services)
    a["date"] = 1000
    a["value"] = encoder.encode(bdu)
    a_res["response"].setComponentByPosition(i, a)

data, _ = decoder.decode(encoder.encode(a_res), asn1Spec=Message())
assert(data.getComponent() == a_res)
assert(data.getComponent().prettyPrint() == a_res.prettyPrint())
print encode(a_res)

Z30TBGViYXkGAikBAgEBMG5pNRMHYUB4LmNvbRMHYkB4LmNvbRMGbGV0dGVyAgID6DAVCQUDMUUtMQkFAzJFLTEJBQM3RS0xaTUTB2NAeC5jb20TB2FAeC5jb20TBnJlbnRlcgICA+gwFQkFAzFFLTEJBQMyRS0xCQUDN0UtMQ==
